In [1]:
#Intall necessary modules and import required packages
import nltk
import random
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
#To parse the JSON file into python we used JSON package
data = open('intents.json').read()
intents = json.loads(data)
all_words=[]
all_classes = []
all_documents = []
ignore_words = ['?', '!']

In [3]:
def processing_data(pattern):
   #fetch patterns and tokenize them into words
   self.pattern=list(map(lambda x:x["patterns"],self.intents))
   self.words=list(map(word_tokenize,flatten(self.pattern)))

In [4]:
#preprocessing the data
#before making a model we pwerformed various preprocessing techniques
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #By fetching the patters we tokenized the data: using nltk.word_tokenize().
        wd_tokens = nltk.word_tokenize(pattern)
        all_words.extend(wd_tokens)
        #append words in the word list
        all_documents.append((wd_tokens, intent['tag']))

        #we created list of classes  for tags
        if intent['tag'] not in all_classes:
            all_classes.append(intent['tag'])

In [5]:
# We lemmatized and lowered each word and removed duplicates by creating a pickle file
all_words = [lemmatizer.lemmatize(wd_tokens.lower()) for wd_tokens in all_words if wd_tokens not in ignore_words]
all_words = sorted(list(set(all_words)))
# all classes are sorted
all_classes = sorted(list(set(all_classes)))
# documents is combination between intents and patterns
print (len(all_documents), "all_documents")
# now classes = intents
print (len(all_classes), "all_classes", all_classes)
# words is all words and vocabulary
print (len(all_words), "unique lemmatized words", all_words)


pickle.dump(all_words,open('all_words.pkl','wb'))
pickle.dump(all_classes,open('all_classes.pkl','wb'))


426 all_documents
68 all_classes ['ABM procedures', 'Academic calendar', 'Admissions', 'Alumni', 'CAAS Staff', 'CAAS services', 'CECIL C. HUMPHREYS SCHOOL OF LAW', 'Canvas', 'Combination Senior Program Guide', 'Contact number', 'Course', 'Current students', 'Explore UofM', 'Faculty and staff', 'Fees', 'Grades', 'Graduate Certificate Programs', 'Graduate Non-Degree Status', 'Graduate catalog', 'Holds', 'Hours of operation', 'International admissions', 'Libraries', 'Location', 'More details about university of memphis', 'Mymemphis', 'Nationally Ranked Graduate Programs', 'News and events', 'Registration', 'Reverse transfer', 'Tiger help request', 'Transcripts', 'University ID', 'University Residence', 'about', 'academic advisor', 'admission deadlines', 'apply to UofM', 'attendance', 'campus visits and events', 'center for athletic academic services', 'commencement', 'complaint', 'contact graduation analyst', 'degree programs', 'financial aid', 'freshman application fee waiver', 'goodbye'

In [6]:
# We created training and test data
training_data = []
# We created empty array for output
empty_array = [0] * len(all_classes)
# for each sentence we initialized bag of words and training set
for doc in all_documents:
    # we initialized bag of words
    bag = []
    # For the pattern - list of tokenized words
    pattern_words = doc[0]
    # To represent related words-  lemmatize each word and create base word, 
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # we created bag of words array with 1, if word found in the current pattern
    for wd_tokens in all_words:
        bag.append(1) if wd_tokens in pattern_words else bag.append(0)
    
    # The output will be '0' for each tag and '1' for current tag for each pattern
    output_row = list(empty_array)
    output_row[all_classes.index(doc[1])] = 1
    
    training_data.append([bag, output_row])
# we shuffled features and turned them into np.array
random.shuffle(training_data)
training_data = np.array(training_data)
# and then we created train and test lists. X as patterns, Y as intents
train_x = list(training_data[:,0])
train_y = list(training_data[:,1])
print("created training data")

#Model building

# we created a model which is a deep neural network with three layers. First layer has 128 neurons, second layer has 64 neurons and third is output layer which containes contains number of neurons
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model. This model give good results.
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#We finally fit and saved the modelas chatbot_model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('uofmchatbot_model', hist)

print("model is created and saved as uofmchatbot_model")

created training data


C:\Users\vijay\AppData\Local\Temp\ipykernel_12676\3473384675.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.array(training_data)
C:\Users\vijay\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
86/86 [==============================] - 1s 3ms/step - loss: 4.1977 - accuracy: 0.0211
Epoch 2/200
86/86 [==============================] - 0s 3ms/step - loss: 4.0884 - accuracy: 0.0634
Epoch 3/200
86/86 [==============================] - 0s 2ms/step - loss: 3.9507 - accuracy: 0.0775
Epoch 4/200
86/86 [==============================] - 0s 2ms/step - loss: 3.7885 - accuracy: 0.1197
Epoch 5/200
86/86 [==============================] - 0s 2ms/step - loss: 3.5683 - accuracy: 0.1620
Epoch 6/200
86/86 [==============================] - 0s 2ms/step - loss: 3.3334 - accuracy: 0.1995
Epoch 7/200
86/86 [==============================] - 0s 2ms/step - loss: 3.1388 - accuracy: 0.2441
Epoch 8/200
86/86 [==============================] - 0s 2ms/step - loss: 2.8973 - accuracy: 0.2958
Epoch 9/200
86/86 [==============================] - 0s 2ms/step - loss: 2.7399 - accuracy: 0.3239
Epoch 10/200
86/86 [==============================] - 0s 2ms/step - loss: 2.6230 - accuracy: 0.3685
Epoch 11/

86/86 [==============================] - 0s 2ms/step - loss: 0.4921 - accuracy: 0.8333
Epoch 84/200
86/86 [==============================] - 0s 2ms/step - loss: 0.4533 - accuracy: 0.8380
Epoch 85/200
86/86 [==============================] - 0s 2ms/step - loss: 0.5832 - accuracy: 0.8239
Epoch 86/200
86/86 [==============================] - 0s 2ms/step - loss: 0.5667 - accuracy: 0.8122
Epoch 87/200
86/86 [==============================] - 0s 2ms/step - loss: 0.4260 - accuracy: 0.8451
Epoch 88/200
86/86 [==============================] - 0s 2ms/step - loss: 0.5528 - accuracy: 0.8122
Epoch 89/200
86/86 [==============================] - 0s 2ms/step - loss: 0.4375 - accuracy: 0.8545
Epoch 90/200
86/86 [==============================] - 0s 2ms/step - loss: 0.3547 - accuracy: 0.8592
Epoch 91/200
86/86 [==============================] - 0s 2ms/step - loss: 0.4022 - accuracy: 0.8521
Epoch 92/200
86/86 [==============================] - 0s 2ms/step - loss: 0.3651 - accuracy: 0.8709
Epoch 93/200


86/86 [==============================] - 0s 2ms/step - loss: 0.4599 - accuracy: 0.8545
Epoch 165/200
86/86 [==============================] - 0s 2ms/step - loss: 0.4099 - accuracy: 0.8474
Epoch 166/200
86/86 [==============================] - 0s 2ms/step - loss: 0.3595 - accuracy: 0.8638
Epoch 167/200
86/86 [==============================] - 0s 2ms/step - loss: 0.3860 - accuracy: 0.8732
Epoch 168/200
86/86 [==============================] - 0s 2ms/step - loss: 0.3526 - accuracy: 0.8850
Epoch 169/200
86/86 [==============================] - 0s 3ms/step - loss: 0.3710 - accuracy: 0.8803
Epoch 170/200
86/86 [==============================] - 0s 2ms/step - loss: 0.3800 - accuracy: 0.8521
Epoch 171/200
86/86 [==============================] - 0s 2ms/step - loss: 0.2593 - accuracy: 0.8873
Epoch 172/200
86/86 [==============================] - 0s 3ms/step - loss: 0.3467 - accuracy: 0.8709
Epoch 173/200
86/86 [==============================] - 0s 2ms/step - loss: 0.3711 - accuracy: 0.8920
Epoc